In [15]:
# CELL 1: Setup - Run this first!
# Enable interactive 3D plots
%matplotlib widget

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from sunpy.coordinates import get_horizons_coord

# Static widgets
event_name = widgets.Text(
    value='MULTI_2024-03-23_01',
    description='Event Name:',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='500px')
)

spacecraft = widgets.Dropdown(
    options=['PSP', 'Solar Orbiter', 'STEREO-A', 'Wind'],
    value='Wind',
    description='Spacecraft:',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='500px')
)

analysis_mode = widgets.RadioButtons(
    options=['CME Event', 'Solar Wind'],
    value='CME Event',
    description='Analysis Mode:',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='500px')
)

# Create separate widget sets for each mode
# CME Mode widgets
cme_start = widgets.Text(
    value='2024-03-24T14:14:00Z',
    description='CME Start:',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='500px')
)

mo_start = widgets.Text(
    value='2024-03-24T18:22:00Z',
    description='MO Start:',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='500px')
)

cme_end = widgets.Text(
    value='2024-03-25T10:04:00Z',
    description='CME End:',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='500px')
)

# Solar Wind Mode widgets
sw_start = widgets.Text(
    value='2023-03-24T00:00:00Z',  # Reasonable default
    description='Start Time:',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='500px')
)

sw_end = widgets.Text(
    value='2023-03-25T00:00:00Z',  # 1 day default
    description='End Time:',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='500px')
)

# Container for time widgets
cme_time_box = widgets.VBox([
    widgets.HTML("<h3>🕐 Time Boundaries</h3>"),
    cme_start,
    mo_start,
    cme_end
])

sw_time_box = widgets.VBox([
    widgets.HTML("<h3>🕐 Time Range</h3>"),
    sw_start,
    sw_end,
    widgets.HTML(
        "<p style='color:#666;font-size:0.9em;margin-top:5px;'>"
        "💡 Tip: Select a time range for solar wind analysis (max 3 days)</p>"
    )
])

time_widgets_container = widgets.VBox()


# Position file (always shown)
pos_file = widgets.Text(
    value='WIND_HELIO1HR_POSITION_1739706',
    description='Position Data:',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='600px')
)

plot_spatial = widgets.Checkbox(
    value=True,
    description='Spatial ATHARV',
    style={'description_width': 'initial'}
)

plot_temporal = widgets.Checkbox(
    value=True,
    description='Temporal ATHARV',
    style={'description_width': 'initial'}
)

plot_hodogram = widgets.Checkbox(
    value=True,
    description='Hodogram (Chinese Fan)',
    style={'description_width': 'initial'}
)

plot_angle = widgets.Checkbox(
    value=True,
    description='Angle vs Time',
    style={'description_width': 'initial'}
)

def update_time_widgets(change=None):
    if analysis_mode.value == 'CME Event':
        time_widgets_container.children = [cme_time_box]
    else:
        time_widgets_container.children = [sw_time_box]

analysis_mode.observe(update_time_widgets, names='value')
update_time_widgets()  # initialize

# Initialize with CME mode display
#update_time_widgets({'new': 'CME Event'})

# ============================================================================
# REMAPPING LOGIC SELECTION (conditional on Spatial ATHARV)
# ============================================================================

remapping_logic = widgets.RadioButtons(
    options=['Expansion-corrected remapping (recommended)', 'Simple remapping'],
    value='Expansion-corrected remapping (recommended)',
    description='Spatial Method:',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='600px')
)

remapping_info = widgets.HTML(
    value="""
    <div style="background-color: #f0f8ff; padding: 10px; border-left: 4px solid #1e88e5; margin: 10px 0;">
        <b>ℹ️ Spatial ATHARV Remapping Methods:</b>
        <ul style="margin: 5px 0;">
            <li><b>Expansion-corrected:</b> Accounts for CME expansion dynamics in spatial reconstruction. More accurate but may fail in some cases.</li>
            <li><b>Simple:</b> Uses direct ∫V dt integration for spatial positions. More robust but overestimates lengths for expanding structures.</li>
        </ul>
        <i>Note: If expansion-corrected fails (singularity detected), automatic fallback to simple remapping will occur.</i>
    </div>
    """
)

remapping_container = widgets.VBox([remapping_logic, remapping_info])

def update_remapping_visibility(change=None):
    """Show/hide remapping options based on mode and Spatial ATHARV selection"""
    show_remapping = (
        plot_spatial.value and 
        analysis_mode.value == 'CME Event'
    )
    
    if show_remapping:
        remapping_container.layout.visibility = 'visible'
        remapping_container.layout.height = 'auto'
        # Update the info text to clarify expansion-corrected needs MO start
        remapping_info.value = """
        <div style="background-color: #f0f8ff; padding: 10px; border-left: 4px solid #1e88e5; margin: 10px 0;">
            <b>ℹ️ Spatial ATHARV Remapping Methods:</b>
            <ul style="margin: 5px 0;">
                <li><b>Expansion-corrected:</b> Accounts for CME expansion dynamics. Requires MO start time.</li>
                <li><b>Simple:</b> Uses direct ∫V dt integration. More robust for all cases.</li>
            </ul>
            <i>Note: If expansion-corrected fails, automatic fallback to simple remapping will occur.</i>
        </div>
        """
    else:
        remapping_container.layout.visibility = 'hidden'
        remapping_container.layout.height = '0px'

# Attach observers
plot_spatial.observe(update_remapping_visibility, names='value')
analysis_mode.observe(update_remapping_visibility, names='value')

# Set initial visibility
remapping_container.layout.visibility = 'visible' if plot_spatial.value else 'hidden'
remapping_container.layout.height = 'auto' if plot_spatial.value else '0px'


# Video Export
save_video = widgets.Checkbox(
    value=False,
    description='Save animations of the 3D plots as MP4 files',
    style={'description_width': 'initial'}
)

# Output widget for dynamic content
dynamic_output = widgets.Output()

# Output widget for results
output = widgets.Output()

# Run Button
run_button = widgets.Button(
    description='Generate Visualizations',
    button_style='success',
    layout=widgets.Layout(width='300px', height='45px'),
    style={'font_weight': 'bold'}
)

# Dictionary to store current data file widgets
current_data_widgets = {}

# Default examples for each spacecraft
SPACECRAFT_DEFAULTS = {
    'STEREO-A': {
        'magplasma_file': 'STA_L2_MAGPLASMA_1M_1734342.csv',
        'event_id': 'MULTI_2024-03-23_01',
        'cme_start': '2024-03-24T14:23:00Z',
        'mo_start': '2024-03-24T19:32:00Z',
        'cme_end': '2024-03-26T06:31:00Z',
        'pos_file': 'STA_HELIO1HR_POSITION_1734342.csv', 
        'sw_start': '2023-03-24T00:00:00Z',
        'sw_end':   '2023-03-25T00:00:00Z',
    },
    'PSP': {
        'mag_file': 'PSP_FLD_L2_MAG_RTN_1MIN_3031590.csv',
        'vel_file': 'PSP_SWP_SPC_L3I_3031590.csv',
        'event_id': 'MULTI_2023-09-22_02',
        'cme_start': '2023-09-22T18:15:00Z',
        'mo_start': '2023-09-22T20:32:00Z',
        'cme_end': '2023-09-22T22:41:00Z',
        'pos_file': 'PSP_HELIO1HR_POSITION_3031590.csv', 
        'sw_start': '2024-03-24T00:00:00Z',
        'sw_end':   '2024-03-25T00:00:00Z',
    },
    'Solar Orbiter': {
        'mag_file': 'SOLO_L2_MAG-RTN-NORMAL-1-MINUTE_1727420.csv',
        'vel_file': 'SOLO_L2_SWA-PAS-GRND-MOM_1727420.csv',
        'event_id': 'MULTI_2024-03-23_01',
        'cme_start': '2024-03-23T13:32:00Z',
        'mo_start': '2024-03-23T15:05:00Z',
        'cme_end': '2024-03-24T02:10:00Z',
        'pos_file': 'SOLO_HELIO1HR_POSITION_1727420.csv', 
        'sw_start': '2023-03-24T00:00:00Z',
        'sw_end':   '2023-03-25T00:00:00Z',
    },
    'Wind': {
        'mag_file': 'WI_H3-RTN_MFI_1739706.csv',
        'vel_file': 'WI_H1_SWE_RTN_1739706.csv',
        'event_id': 'MULTI_2024-03-23_01',
        'cme_start': '2024-03-24T14:14:00Z',
        'mo_start': '2024-03-24T18:22:00Z',
        'cme_end': '2024-03-25T10:04:00Z',
        'pos_file': 'WIND_HELIO1HR_POSITION_1739706.csv', 
        'sw_start': '2023-03-24T00:00:00Z',
        'sw_end':   '2023-03-25T00:00:00Z',
    }
}

def update_data_files(change):
    """Update data file widgets based on spacecraft selection"""
    sc = change['new']
    defaults = SPACECRAFT_DEFAULTS[sc]

    if analysis_mode.value == 'CME Event':
        cme_start.value = defaults['cme_start']
        mo_start.value  = defaults['mo_start']
        cme_end.value   = defaults['cme_end']

    elif analysis_mode.value == 'Solar Wind':
        sw_start.value = defaults.get('sw_start', sw_start.value)
        sw_end.value   = defaults.get('sw_end', sw_end.value)
    
    with dynamic_output:
        clear_output(wait=True)
        
        if sc == 'STEREO-A':
            # STEREO-A uses combined magplasma file
            magplasma_file = widgets.Text(
                value=defaults['magplasma_file'],
                description='Magplasma Data:',
                style={'description_width': '150px'},
                layout=widgets.Layout(width='600px')
            )
            current_data_widgets['magplasma_file'] = magplasma_file
            display(magplasma_file)
            
        else:
            # Other spacecraft use separate mag and vel files
            mag_file = widgets.Text(
                value=defaults['mag_file'],
                description='Magnetic Field:',
                style={'description_width': '150px'},
                layout=widgets.Layout(width='600px')
            )
            vel_file = widgets.Text(
                value=defaults['vel_file'],
                description='Velocity Data:',
                style={'description_width': '150px'},
                layout=widgets.Layout(width='600px')
            )
            current_data_widgets['mag_file'] = mag_file
            current_data_widgets['vel_file'] = vel_file
            display(mag_file)
            display(vel_file)
    
    # Update other defaults
    event_name.value = defaults['event_id']
    # cme_start.value = defaults['cme_start']
    # mo_start.value = defaults['mo_start']
    # cme_end.value = defaults['cme_end']
    pos_file.value = defaults['pos_file']

# Attach observer to spacecraft dropdown
spacecraft.observe(update_data_files, names='value')

# Initial display
update_data_files({'new': spacecraft.value})

# # Display all widgets
# display(widgets.HTML("<h3>Event Configuration</h3>"))
# display(event_name)
# display(spacecraft)

# display(widgets.HTML("<h3>Time Boundaries</h3>"))
# display(cme_start)
# display(mo_start)
# display(cme_end)

# display(widgets.HTML("<h3>Data Files</h3>"))
# display(dynamic_output)  # This will show mag/vel or magplasma based on spacecraft
# display(pos_file)

# display(widgets.HTML("<h3>Export Options</h3>"))
# display(save_video)

# display(widgets.HTML("<br>"))
# display(run_button)
# display(output)

# Validation output
validation_output = widgets.Output()
# display(validation_output)

def validate_time_duration(change=None):
    """Validate that time duration doesn't exceed 3 days"""
    from datetime import datetime, timedelta
    
    with validation_output:
        clear_output(wait=True)
        
        try:
            # Get the correct time fields based on mode
            if analysis_mode.value == 'CME Event':
                start_time = cme_start.value
                mo_time = mo_start.value
                end_time = cme_end.value
            else:  # Solar Wind
                start_time = sw_start.value
                mo_time = None
                end_time = sw_end.value

            start = datetime.fromisoformat(start_time.replace('Z', '+00:00'))
            end = datetime.fromisoformat(end_time.replace('Z', '+00:00'))
            mo = datetime.fromisoformat(mo_time.replace('Z', '+00:00')) if mo_time else None

            duration = end - start
            max_duration = timedelta(days=3)

            # ---------- Existing checks ----------
            if duration > max_duration:
                days = duration.total_seconds() / 86400
                print(f"⚠️  Warning: Duration is {days:.2f} days (max allowed: 3 days)")
                print(f"   Please reduce the time range.")
                run_button.disabled = True
                run_button.button_style = 'danger'

            elif duration <= timedelta(0):
                print(f"⚠️  Error: End time must be after start time")
                run_button.disabled = True
                run_button.button_style = 'danger'

            # ---------- NEW CHECK: MO inside CME window ----------
            elif analysis_mode.value == 'CME Event' and mo is not None:
                if not (start <= mo <= end):
                    print("⚠️  Error: MO time must lie between CME start and CME end")
                    run_button.disabled = True
                    run_button.button_style = 'danger'

                else:
                    days = duration.total_seconds() / 86400
                    print(f"✓ Duration: {days:.2f} days")
                    print("✓ MO time is within CME window")
                    run_button.disabled = False
                    run_button.button_style = 'success'

            else:
                days = duration.total_seconds() / 86400
                print(f"✓ Duration: {days:.2f} days")
                run_button.disabled = False
                run_button.button_style = 'success'

        except ValueError as e:
            print(f"⚠️  Error parsing dates: {e}")
            run_button.disabled = True
            run_button.button_style = 'danger'


# Attach validation to all time fields
cme_start.observe(validate_time_duration, names='value')
mo_start.observe(validate_time_duration, names='value')
cme_end.observe(validate_time_duration, names='value')
sw_start.observe(validate_time_duration, names='value')
sw_end.observe(validate_time_duration, names='value')
analysis_mode.observe(validate_time_duration, names='value')

# Run initial validation
validate_time_duration()

# Helper function to get current data file values
def get_data_files():
    """Returns dict of current data file values"""
    return {key: widget.value for key, widget in current_data_widgets.items()}


# =============================================================================
# MAIN PROCESSING FUNCTION
# =============================================================================


def run_visualization(b):
    """Execute CME visualization with widget settings"""
    import matplotlib.pyplot as plt
    with output:
        output.clear_output(wait=True)
        plt.close('all')

        #HARD RESET — prevents state leakage
        time = None
        mask = None
        solar_wind_data = None
        t_start = None
        t_end = None
        
        try:
            # Check if at least one plot is selected
            if not any([plot_spatial.value, plot_temporal.value, plot_hodogram.value, plot_angle.value]):
                print("⚠️  Please select at least one plot to generate!")
                return
            
            # Get configuration
            EVENT = event_name.value
            SPACECRAFT = spacecraft.value
            SAVE_MP4 = save_video.value
            USE_EXPANSION_CORRECTED = remapping_logic.value.startswith('Expansion-corrected')
            
            print("="*70)
            print(f"🎯 Event: {EVENT}")
            print(f"🛸 Spacecraft: {SPACECRAFT}")
            print(f"📊 Plots to generate:")
            if plot_spatial.value:
                method = "Expansion-corrected" if USE_EXPANSION_CORRECTED else "Simple"
                print(f"   • Spatial ATHARV ({method} remapping)")
            if plot_temporal.value:
                print(f"   • Temporal ATHARV")
            if plot_hodogram.value:
                print(f"   • Hodogram")
            if plot_angle.value:
                print(f"   • Angle vs Time")
            print("="*70)
            print()
            
            # ============================================================
            # IMPORTS
            # ============================================================
            print("📦 Loading libraries...")
            from astropy.io import ascii
            import numpy as np
            from datetime import timedelta
            from dateutil import parser
            import solgaleo
            import matplotlib.pyplot as plt
            from matplotlib.colors import Normalize, TwoSlopeNorm, to_rgba
            from matplotlib.cm import ScalarMappable
            from matplotlib.ticker import MaxNLocator
            from matplotlib.collections import LineCollection
            from mpl_toolkits.axes_grid1 import make_axes_locatable
            from astropy.time import Time
            import matplotlib.dates as mdates
            from matplotlib.animation import FuncAnimation
            import subprocess
            import os
            import pandas as pd
            print("✅ Libraries loaded\n")
            
            # ============================================================
            # PARSE TIME CONFIGURATION
            # ============================================================
            #Initialize CDAWeb service
            from cdasws import CdasWs
            import numpy as np
            from dateutil import parser
            import time
            from sunpy.coordinates import get_horizons_coord
            from sunpy.coordinates.frames import HeliocentricInertial
            from astropy.time import Time
            from astropy import units as u

            # Initialize CDAWeb service
            cdas = CdasWs()

            # Spacecraft ID mapping for Horizons
            HORIZONS_IDS = {
                'Wind': 'Wind',
                'STEREO-A': 'STEREO-A',
                'PSP': 'Parker Solar Probe',
                'Solar Orbiter': 'Solar Orbiter'
            }

            def fetch_with_retry(dataset, variables, start_time, end_time, max_retries=3, delay=2):
                """Fetch data from CDAWeb with retry logic"""
                for attempt in range(max_retries):
                    try:
                        result = cdas.get_data(dataset, variables, start_time, end_time)
                        if result and len(result) > 1 and result[1] is not None:
                            return result[1]
                        else:
                            print(f"    Attempt {attempt+1}: No data returned, retrying...")
                            time.sleep(delay)
                    except Exception as e:
                        print(f"    Attempt {attempt+1} failed: {str(e)[:100]}")
                        if attempt < max_retries - 1:
                            time.sleep(delay * (attempt + 1))  # Exponential backoff
                        else:
                            raise
                raise Exception(f"Failed to fetch data from {dataset} after {max_retries} attempts")

            def get_spacecraft_position(spacecraft, start_time, end_time):
                """Get spacecraft position using SunPy/Horizons for hourly resolution"""
                print(f"  Fetching {spacecraft} position from Horizons...")
                
                # Create hourly time array
                start_astropy = Time(start_time)
                end_astropy = Time(end_time)
                duration_hours = int((end_astropy - start_astropy).to(u.hour).value) + 1
                times = start_astropy + np.arange(duration_hours) * u.hour
                
                # Get coordinates from Horizons in HGI frame
                coords = get_horizons_coord(HORIZONS_IDS[spacecraft], times)
                
                # Convert to HGI (Heliographic Inertial) if not already
                coords_hgi = coords.transform_to(HeliocentricInertial())
                
                # Extract position components
                t_R_sc = times.datetime  # Convert to datetime objects
                R_sc_r = coords_hgi.distance.to(u.km).value  # Already in km
                R_sc_lon = coords_hgi.lon.to(u.deg).value  # Longitude in degrees
                R_sc_lat = coords_hgi.lat.to(u.deg).value  # Latitude in degrees
                
                return t_R_sc, R_sc_r, R_sc_lon, R_sc_lat

            if analysis_mode.value == 'CME Event':
                T_CME_START = parser.isoparse(cme_start.value)
                T_MO_START = parser.isoparse(mo_start.value)
                T_CME_END = parser.isoparse(cme_end.value)
                USE_EXPANSION_CORRECTED = (
                    remapping_logic.value.startswith('Expansion-corrected') and 
                    plot_spatial.value
                )
            else:  # Solar Wind
                T_CME_START = parser.isoparse(sw_start.value)
                T_MO_START = None  # Not used for solar wind
                T_CME_END = parser.isoparse(sw_end.value)
                USE_EXPANSION_CORRECTED = False  # Always use simple for solar wind
            
            AU = 149597870.7  # km

            print("📂 Loading data from CDAWeb...")

            print("Time duration:", T_CME_START, "→", T_CME_END)


            # Fetch data based on spacecraft
            if SPACECRAFT == 'Wind':
                # Magnetic field data
                print("  Fetching Wind magnetic field...")
                mag_data = fetch_with_retry('WI_H3-RTN_MFI', ['BRTN'], T_CME_START, T_CME_END)
                t_B = mag_data['Epoch']
                B_rtn = mag_data['BRTN']
                
                time.sleep(1)  # Brief pause between requests
                
                if plot_spatial.value:
                    # Velocity data
                    print("  Fetching Wind velocity...")
                    vel_data = fetch_with_retry('WI_H1_SWE_RTN', 
                                                ['Proton_VR_moment', 'Proton_VT_moment', 'Proton_VN_moment'], 
                                                T_CME_START, T_CME_END)
                    t_V = vel_data['Epoch']
                    V_rtn = np.column_stack((
                        vel_data['Proton_VR_moment'],
                        vel_data['Proton_VT_moment'],
                        vel_data['Proton_VN_moment']
                    ))
                
                    time.sleep(1)  # Brief pause between requests
                    
                    # Position data from Horizons
                    t_R_sc, R_sc_r, R_sc_lon, R_sc_lat = get_spacecraft_position('Wind', T_CME_START, T_CME_END)

            elif SPACECRAFT == 'STEREO-A':
                # Combined magnetic field and velocity data
                print("  Fetching STEREO-A mag/plasma...")
                combined_data = fetch_with_retry('STA_L2_MAGPLASMA_1M',
                                                ['BFIELDRTN', 'Vp'],
                                                T_CME_START, T_CME_END)
                t_B = combined_data['Epoch']
                B_rtn = combined_data['BFIELDRTN']
                
                if plot_spatial.value:
                    # Velocity - only bulk speed, so put in R component with T,N as zeros
                    t_V = t_B
                    V_r = combined_data['Vp']
                    V_t = np.zeros(len(V_r))
                    V_n = np.zeros(len(V_r))
                    V_rtn = np.column_stack((V_r, V_t, V_n))
                    
                    time.sleep(1)  # Brief pause between requests
                    
                    # Position data from Horizons
                    t_R_sc, R_sc_r, R_sc_lon, R_sc_lat = get_spacecraft_position('STEREO-A', T_CME_START, T_CME_END)

            elif SPACECRAFT == 'PSP':
                # Magnetic field data
                print("  Fetching PSP magnetic field...")
                mag_data = fetch_with_retry('PSP_FLD_L2_MAG_RTN_1MIN',
                                            ['psp_fld_l2_mag_RTN_1min'],
                                            T_CME_START, T_CME_END)
                t_B = mag_data['epoch_mag_RTN_1min']
                B_rtn = mag_data['psp_fld_l2_mag_RTN_1min']
                
                time.sleep(1)  # Brief pause between requests
                
                if plot_spatial.value:
                    # Velocity data
                    print("  Fetching PSP velocity...")
                    vel_data = fetch_with_retry('PSP_SWP_SPC_L3I',
                                                ['vp_moment_RTN'],
                                                T_CME_START, T_CME_END)
                    t_V = vel_data['Epoch']
                    V_rtn = vel_data['vp_moment_RTN']
                    
                    time.sleep(1)  # Brief pause between requests
                    
                    # Position data from Horizons
                    t_R_sc, R_sc_r, R_sc_lon, R_sc_lat = get_spacecraft_position('PSP', T_CME_START, T_CME_END)

            elif SPACECRAFT == 'Solar Orbiter':
                # Magnetic field data
                print("  Fetching Solar Orbiter magnetic field...")
                mag_data = fetch_with_retry('SOLO_L2_MAG-RTN-NORMAL-1-MINUTE',
                                            ['B_RTN'],
                                            T_CME_START, T_CME_END)
                t_B = mag_data['EPOCH']
                B_rtn = mag_data['B_RTN']
                
                time.sleep(1)  # Brief pause between requests
                
                if plot_spatial.value:
                    # Velocity data
                    print("  Fetching Solar Orbiter velocity...")
                    vel_data = fetch_with_retry('SOLO_L2_SWA-PAS-GRND-MOM',
                                                ['V_RTN'],
                                                T_CME_START, T_CME_END)
                    t_V = vel_data['Epoch']
                    V_rtn = vel_data['V_RTN']
                    
                    time.sleep(1)  # Brief pause between requests
                    
                    # Position data from Horizons
                    t_R_sc, R_sc_r, R_sc_lon, R_sc_lat = get_spacecraft_position('Solar Orbiter', T_CME_START, T_CME_END)


            # Convert xarray datetime64 to Python datetime objects
            #t_B = [pd.Timestamp(t.values).to_pydatetime() for t in t_B]
            
            #if plot_spatial.value:
            #    t_V = [pd.Timestamp(t.values).to_pydatetime() for t in t_V]
            
            print("✅ Data loading complete!")

            if plot_spatial.value:
                theta = 90 - np.array(R_sc_lat)
                R_sc_hgi = solgaleo.batch_spherical_to_cartesian(R_sc_r, theta, R_sc_lon, degrees=True)
                t_V = [t.replace(tzinfo=None) for t in t_V]

            t_B = [t.replace(tzinfo=None) for t in t_B]

            T_CME_START = T_CME_START.replace(tzinfo=None)
            if analysis_mode.value == 'CME Event':
                T_MO_START = T_MO_START.replace(tzinfo=None)
            T_CME_END = T_CME_END.replace(tzinfo=None)

            # DATA CLIPPING TO CME INTERVAL
            print("\n✂️ Clipping data to CME interval...")
            idx_cme_start_orig = np.argmin([abs(t - T_CME_START) for t in t_B])
            if analysis_mode.value == 'CME Event':
                idx_mo_start_orig = np.argmin([abs(t - T_MO_START) for t in t_B])
            idx_cme_end_orig = np.argmin([abs(t - T_CME_END) for t in t_B])
            t_B = [t.values if hasattr(t, 'values') else t for t in t_B[idx_cme_start_orig:idx_cme_end_orig+1]]
            B_rtn = B_rtn[idx_cme_start_orig:idx_cme_end_orig+1]

            if plot_spatial.value:
                idx_v_start = np.argmin([abs(t - T_CME_START) for t in t_V])
                idx_v_end = np.argmin([abs(t - T_CME_END) for t in t_V])
                t_V = [t.values if hasattr(t, 'values') else t for t in t_V[idx_v_start:idx_v_end+1]]
                V_rtn = V_rtn[idx_v_start:idx_v_end+1]
                idx_r_start = np.argmin([abs(t - T_CME_START) for t in t_R_sc])
                idx_r_end = np.argmin([abs(t - T_CME_END) for t in t_R_sc])
                t_R_sc = t_R_sc[idx_r_start:idx_r_end+1]
                R_sc_hgi = R_sc_hgi[idx_r_start:idx_r_end+1]


            # QUALITY FILTERING
            print("\n🔍 Filtering...")
            B_bounds = [(-1e4, 1e4)] * 3
            B_mask = solgaleo.mask_vectors_by_components(B_rtn, B_bounds)
            t_B = [t for t, ok in zip(t_B, B_mask) if ok]
            B_rtn = np.asarray(B_rtn[B_mask], dtype=float)

            if plot_spatial.value:
                V_bounds = [(-1e4, 1e4), (-1e4, 1e4), (-1e4, 1e4)]
                R_bounds = [(-1e10, 1e10)] * 3          
                V_mask = solgaleo.mask_vectors_by_components(V_rtn, V_bounds)
                R_mask = solgaleo.mask_vectors_by_components(R_sc_hgi, R_bounds)
                t_V = [t for t, ok in zip(t_V, V_mask) if ok]
                V_rtn_clean = np.asarray(V_rtn[V_mask], dtype=float)
                t_R_sc = [t for t, ok in zip(t_R_sc, R_mask) if ok]
                R_sc_hgi = np.asarray(R_sc_hgi[R_mask], dtype=float)

            # INTERPOLATION
            if USE_EXPANSION_CORRECTED:
                print("\n🔄 Interpolating...")

            if plot_spatial.value:
                V_rtn_interp, t_B_aligned = solgaleo.interpolate_vector_series(
                    t_V, V_rtn_clean, t_B, align=True
                )
                R_sc_hgi_interp, _ = solgaleo.pchip_vector_interp(
                    t_R_sc, R_sc_hgi, t_B_aligned, align=True
                )

                # Align B data with interpolated times
                mask_B_aligned = np.array([
                    any(abs((t - t_aligned).total_seconds()) < 0.1 for t_aligned in t_B_aligned) 
                    for t in t_B
                ])
                t_B = t_B_aligned
                B_rtn = B_rtn[mask_B_aligned]

                # Ensure all arrays have the same length
                min_len = min(len(V_rtn_interp), len(R_sc_hgi_interp), len(t_B), len(B_rtn))
                V_rtn_interp = V_rtn_interp[:min_len]
                R_sc_hgi_interp = R_sc_hgi_interp[:min_len]
                t_B = t_B[:min_len]
                B_rtn = B_rtn[:min_len]


            # COORDINATE TRANSFORMATIONS
            
            if plot_spatial.value:
                print("\n🧭 Transforming coordinates...")
                V_hgi = solgaleo.batch_rtn_to_hgi(V_rtn_interp, R_sc_hgi_interp)
                B_hgi = solgaleo.batch_rtn_to_hgi(B_rtn, R_sc_hgi_interp)
            B_mag = np.sqrt(B_rtn[:, 0]**2 + B_rtn[:, 1]**2 + B_rtn[:, 2]**2)
            
            idx_cme_start = np.argmin([abs(t - T_CME_START) for t in t_B])
            if analysis_mode.value == 'CME Event':
                idx_mo_start = np.argmin([abs(t - T_MO_START) for t in t_B])
            idx_cme_end = np.argmin([abs(t - T_CME_END) for t in t_B])
            

            # Only do velocity fitting if spatial plot is selected
            if plot_spatial.value:
                if USE_EXPANSION_CORRECTED:
                    print("\nUsing expansion-corrected remapping...")
                    print("\n📈 Fitting velocities...")
                    
                    v_le_mo, v_te_mo, v_cm_mo, v_exp_mo = \
                        solgaleo.fit_velocity_edges(
                            t_B, V_hgi, T_MO_START, T_CME_END,
                            plot=False, 
                            #title="MO Region Velocity Fit"
                        )
                    
                    # CME POSITION REMAPPING
                    print("\n🗺️ Remapping positions...")
                    
                    remapping_failed = False
                    try:
                        R_remapped, is_sheath = solgaleo.calculate_cme_positions(
                            t_B, R_sc_hgi_interp, T_CME_START, T_MO_START, T_CME_END,
                            V_hgi, v_cm_mo, v_exp_mo
                        )
                        R_remapped_rotated = solgaleo.custom_rotate_frame(R_remapped, R_remapped[idx_cme_start])
                        
                        # Check for singularities (NaN, Inf, or large values)
                        if (
                            not np.all(np.isfinite(R_remapped)) or
                            np.ptp(R_remapped_rotated[:, 0]) / AU > 1.0 or
                            np.ptp(R_remapped_rotated[:, 1]) / AU > 0.1 or
                            np.ptp(R_remapped_rotated[:, 2]) / AU > 0.1
                        ):
                            print("  ⚠️  Singularity detected in expansion-corrected remapping!")
                            remapping_failed = True
                    except Exception as e:
                        print(f"  ⚠️  Expansion-corrected remapping failed: {e}")
                        remapping_failed = True
                    
                    if remapping_failed:
                        print("  ↩️  Falling back to simple remapping...")
                        USE_EXPANSION_CORRECTED = False
                
                if not USE_EXPANSION_CORRECTED or remapping_failed:
                    print("  Using simple remapping...")
                    
                    dt = np.diff(t_B)
                    dt = np.array([d.total_seconds() for d in dt])
                    dt = np.append(dt, dt[-1])  # keep length = N

                    Vdt = V_hgi * dt[:, np.newaxis]  # shape (N, 3)
                    displacement = np.cumsum(Vdt, axis=0)
                    R_remapped = R_sc_hgi_interp - displacement
                    R_remapped_rotated = solgaleo.custom_rotate_frame(R_remapped, R_remapped[idx_cme_start])
                    #midpoint = (np.max(R_remapped, axis=0) + np.min(R_remapped, axis=0)) / 2
                    #R_remapped_rotated = solgaleo.custom_rotate_frame(R_remapped, midpoint)
                
                
                #B_hgi_rotated = solgaleo.custom_rotate_frame(B_hgi, R_remapped[idx_cme_start])
                B_hgi_rotated = solgaleo.custom_rotate_frame(B_hgi, R_remapped[idx_cme_start+idx_cme_end//2])
            
            # print("Spacecraft:", spacecraft.value)
            # print("Time range:", time.min(), time.max())
            # print("Masked points:", mask.sum())
            # print("First 3 values:", data['speed'][mask][:3])

            # ============================================================================
            # SPATIAL REMAPPED VISUALIZATION
            # ============================================================================
            if plot_spatial.value:
                print("\n🎨 Creating spatial remapped plot...")

                X = R_remapped_rotated[:, 0] / AU
                Y = R_remapped_rotated[:, 1] / AU
                Z = R_remapped_rotated[:, 2] / AU
                Bx, By, Bz = B_hgi_rotated[:, 0], B_hgi_rotated[:, 1], B_hgi_rotated[:, 2]

                # Apply positive X mask
                mask_posX = X >= 0
                X = X[mask_posX]
                Y = Y[mask_posX]
                Z = Z[mask_posX]
                Bx = Bx[mask_posX]
                By = By[mask_posX]
                Bz = Bz[mask_posX]
                B_mag_plot = B_mag[mask_posX]
                t_B_plot = [t for i, t in enumerate(t_B) if mask_posX[i]]

                if len(X) == 0:
                    raise ValueError("No data points with X >= 0. Cannot create plot.")

                # Normalize for color mapping
                norm_mag = Normalize(vmin=np.nanmin(B_mag_plot), vmax=np.nanmax(B_mag_plot))
                normed_mag = np.clip(norm_mag(B_mag_plot), 0, 1)

                # Plot setup
                fig = plt.figure(figsize=(12, 7))
                ax = fig.add_subplot(111, projection='3d')

                # Dimension ratios
                xlen, ylen, zlen = 4, 1, 1
                xrange = 1.1 * (X.max() - X.min())
                yrange = xrange * ylen / xlen
                zrange = xrange * zlen / xlen
                xlim = ((np.abs(X).max() + np.abs(X).min()) / 2 - xrange / 2,
                        (np.abs(X).max() + np.abs(X).min()) / 2 + xrange / 2)
                ylim = -yrange / 2, yrange / 2
                zlim = -zrange / 2, zrange / 2
                B_scale_factor = 0.707 * yrange / np.max(B_mag_plot)

                cmap = plt.colormaps.get_cmap('viridis')

                # Plot quiver arrows
                for i in range(len(X)):
                    if np.isnan(normed_mag[i]):
                        continue
                    u = Bx[i] * B_scale_factor
                    v = By[i] * B_scale_factor
                    w = Bz[i] * B_scale_factor
                    rgba = to_rgba(cmap(normed_mag[i]), alpha=1)
                    ax.quiver(X[i], Y[i], Z[i], u, v, w, color=rgba, 
                            arrow_length_ratio=0.1, linewidth=1.2)

                # Overlay scatter for B_R
                cmap_br = 'seismic'
                br_norm = TwoSlopeNorm(
                    vmin=-max(abs(np.nanmin(Bx)), abs(np.nanmax(Bx))),
                    vcenter=0,
                    vmax=max(abs(np.nanmin(Bx)), abs(np.nanmax(Bx)))
                )
                y_offset = 0
                z_offset = -zrange / 2 * 0.9

                sc = ax.scatter(X, Y + y_offset, Z + z_offset, c=Bx, cmap=cmap_br, 
                                norm=br_norm, s=0.5, alpha=1)

                # Time labels
                time_seconds = np.array([(t - t_B_plot[0]).total_seconds() for t in t_B_plot])
                X_arr = np.array(X, dtype=float)
                sort_idx = np.argsort(X_arr)
                X_unique, idx = np.unique(X_arr[sort_idx], return_index=True)
                time_unique = time_seconds[sort_idx][idx]

                ax.xaxis.set_major_locator(MaxNLocator(nbins=8))
                r0_ticks = ax.get_xticks()
                r0_ticks = r0_ticks[(r0_ticks >= xlim[0]) & (r0_ticks <= xlim[1])]

                n_labels = 8
                tick_positions = r0_ticks[np.linspace(0, len(r0_ticks)-1, n_labels, dtype=int)] \
                    if len(r0_ticks) > n_labels else r0_ticks

                tick_seconds = np.interp(tick_positions, X_unique, time_unique)
                tick_times = [t_B_plot[0] + timedelta(seconds=s) for s in tick_seconds]

                axis_y = 0
                axis_z = zrange / 2 * 1.6

                for x_pos, t in zip(tick_positions, tick_times):
                    ax.text(x_pos, axis_y, axis_z, t.strftime("%m-%d\n%H:%M"),
                            ha="center", va="bottom", fontsize=8, color="gray")

                # Axis formatting
                ax.set_box_aspect([xlen, ylen, zlen])
                ax.set_title(f"{SPACECRAFT} - spatial ATHARV plot - {EVENT}")
                ax.set_xlabel('$R_0$ (AU)', labelpad=25)
                ax.set_ylabel('$T_0$ (AU)', labelpad=8)
                ax.set_zlabel('$N_0$ (AU)')
                ax.set_xlim(xlim[0], xlim[1])
                ax.set_ylim(ylim[0], ylim[1])
                ax.set_zlim(zlim[0], zlim[1])
                ax.yaxis.set_major_locator(MaxNLocator(nbins=3))
                ax.zaxis.set_major_locator(MaxNLocator(nbins=3))

                # Colorbars
                sm_mag = ScalarMappable(cmap=cmap, norm=norm_mag)
                sm_mag.set_array([])
                cbar_mag_ax = fig.add_axes([0.85, 0.4, 0.008, 0.3])
                cbar_mag = fig.colorbar(sm_mag, cax=cbar_mag_ax)
                cbar_mag.set_label('$|B|$ (nT)')

                cbar_br_ax = fig.add_axes([0.15, 0.4, 0.008, 0.3])
                cbar_br = fig.colorbar(sc, cax=cbar_br_ax)
                cbar_br.ax.yaxis.set_ticks_position('left')
                cbar_br.ax.yaxis.set_label_position('left')
                cbar_br.set_label(r'$B_{R_0}$ (nT)')

                if analysis_mode.value == 'CME Event':
                    # Time planes
                    original_to_filtered = {i: len([j for j in range(i) if mask_posX[j]]) 
                                            for i in range(len(mask_posX)) if mask_posX[i]}

                    if idx_cme_start in original_to_filtered:
                        solgaleo.draw_time_plane(ax, X[original_to_filtered[idx_cme_start]], 
                                                color='blue', alpha=0.2)
                    if idx_mo_start in original_to_filtered:
                        solgaleo.draw_time_plane(ax, X[original_to_filtered[idx_mo_start]], 
                                                color='blueviolet', alpha=0.2)
                    if idx_cme_end in original_to_filtered:
                        solgaleo.draw_time_plane(ax, X[original_to_filtered[idx_cme_end]], 
                                                color='red', alpha=0.2)

                    for orig_idx, c in [(idx_cme_start, 'k'), (idx_cme_end, 'k')]:
                        if orig_idx in original_to_filtered:
                            i = original_to_filtered[orig_idx]
                            ax.plot([X[i], X[i]], [Y[i], Y[i]], [ax.get_zlim()[0], Z[i]],
                                    color=c, linestyle='--', linewidth=1, alpha=1)
                    for orig_idx, c in [(idx_mo_start, 'k')]:
                        if orig_idx in original_to_filtered:
                            i = original_to_filtered[orig_idx]
                            ax.plot([X[i], X[i]], [Y[i], Y[i]], [ax.get_zlim()[0], Z[i]],
                                    color=c, linestyle='--', linewidth=1, alpha=1)

                    ax.scatter([], [], [], color='blue', label='CME Start', marker='s', alpha=0.2)
                    ax.scatter([], [], [], color='blueviolet', label='MO Start', marker='s', alpha=0.2)
                    ax.scatter([], [], [], color='red', label='CME End', marker='s', alpha=0.2)
                    ax.legend(loc='upper left')

                fig.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.0)

                plt.show()  # Display the static plot
                
                #print("  ✅ Spatial plot displayed (you can interact with it above)")

                # Create animation only if saving
                if SAVE_MP4:
                    print("  💾 Saving MP4...")
                    def update_spatial(angle):
                        ax.view_init(elev=20, azim=-60 + angle)
                        return fig,

                    ani_spatial = FuncAnimation(fig, update_spatial, frames=range(0, 360, 2), 
                                            interval=50, blit=True)
                    temp_file = f"{EVENT}_{SPACECRAFT}-spatial_ATHARV_temp.mp4"
                    final_file = f"{EVENT}_{SPACECRAFT}-spatial_ATHARV.mp4"
                    import matplotlib as mpl
                    mpl.rcParams['animation.ffmpeg_args'] = ['-loglevel', 'error']

                    ani_spatial.save(
                        temp_file,
                        writer="ffmpeg"
                    )

                    subprocess.run(
                        [
                            'ffmpeg', '-loglevel', 'error',
                            '-i', temp_file,
                            '-vf', 'crop=in_w*0.85:in_h*0.9:in_w*0.075:in_h*0.0',
                            '-c:a', 'copy', final_file
                        ],
                        stdout=subprocess.DEVNULL,
                        stderr=subprocess.DEVNULL,
                        check=True
                    )

                    os.remove(temp_file)
                    print(f"  ✅ Saved: {final_file}")

                    from IPython.display import HTML, display
                    import base64

                    with open(final_file, "rb") as f:
                        video_b64 = base64.b64encode(f.read()).decode("utf-8")

                    display(HTML(f"""
                    <video width="800"
                        autoplay
                        loop
                        muted
                        playsinline
                        controls>
                    <source src="data:video/mp4;base64,{video_b64}" type="video/mp4">
                    </video>

                    <p>
                    <a download="{final_file}"
                        href="data:video/mp4;base64,{video_b64}">
                        ⬇️ Download MP4
                    </a>
                    </p>
                    """))

                    #plt.close(fig_temp)

                else:
                    # Don't close the figure so user can interact with it
                    pass

            # ============================================================================
            # TEMPORAL VISUALIZATION
            # ============================================================================
            if plot_temporal.value:
                print("\n🎨 Creating temporal plot...")

                time_astropy = Time(t_B)
                X_temp = time_astropy.plot_date
                Y_temp = np.zeros_like(X_temp)
                Z_temp = np.zeros_like(X_temp)

                Bx_temp, By_temp, Bz_temp = B_rtn[:, 0], B_rtn[:, 1], B_rtn[:, 2]
                X_temp_all = X_temp.copy()

                # mask_posX_temp = X_temp >= 0
                # X_temp = X_temp[mask_posX_temp]
                # Y_temp = Y_temp[mask_posX_temp]
                # Z_temp = Z_temp[mask_posX_temp]
                # Bx_temp = Bx_temp[mask_posX_temp]
                # By_temp = By_temp[mask_posX_temp]
                # Bz_temp = Bz_temp[mask_posX_temp]
                B_mag_temp = B_mag
                X_duration = X_temp[-1] - X_temp[0]

                norm_mag_temp = Normalize(vmin=np.nanmin(B_mag_temp), vmax=np.nanmax(B_mag_temp))
                normed_mag_temp = np.clip(norm_mag_temp(B_mag_temp), 0, 1)

                fig_temp = plt.figure(figsize=(12, 7))
                ax_temp = fig_temp.add_subplot(111, projection='3d')

                # Dimensions
                xlen_t, ylen_t, zlen_t = 4, 1, 1
                xrange_t = 1.1 * (X_temp_all.max() - X_temp_all.min())
                yrange_t = xrange_t * ylen_t / xlen_t
                zrange_t = xrange_t * zlen_t / xlen_t
                xlim_t = ((np.abs(X_temp_all).max() + np.abs(X_temp_all).min()) / 2 - xrange_t / 2,
                        (np.abs(X_temp_all).max() + np.abs(X_temp_all).min()) / 2 + xrange_t / 2)
                ylim_t = -yrange_t / 2, yrange_t / 2
                zlim_t = -zrange_t / 2, zrange_t / 2
                B_scale_factor_t = 0.707 / np.max(B_mag_temp)

                cmap_temp = plt.colormaps.get_cmap('viridis')

                # Plot quiver arrows
                for i in range(len(X_temp)):
                    if np.isnan(normed_mag_temp[i]):
                        continue
                    u = -Bx_temp[i] * B_scale_factor_t * X_duration / xlen_t
                    v = -By_temp[i] * B_scale_factor_t * yrange_t / ylen_t
                    w = Bz_temp[i] * B_scale_factor_t * zrange_t / zlen_t
                    rgba = to_rgba(cmap_temp(normed_mag_temp[i]), alpha=1)
                    ax_temp.quiver(X_temp[i], Y_temp[i], Z_temp[i], u, v, w, color=rgba,
                                arrow_length_ratio=0.1, linewidth=1.2)

                # Scatter overlay
                br_norm_temp = TwoSlopeNorm(
                    vmin=-max(abs(np.nanmin(Bx_temp)), abs(np.nanmax(Bx_temp))),
                    vcenter=0,
                    vmax=max(abs(np.nanmin(Bx_temp)), abs(np.nanmax(Bx_temp)))
                )
                y_offset_t = -yrange_t / 2 * 0.9
                z_offset_t = -zrange_t / 2 * 0.9

                sc_temp = ax_temp.scatter(X_temp, Y_temp + y_offset_t, Z_temp + z_offset_t,
                                        c=Bx_temp, cmap='seismic', norm=br_norm_temp, 
                                        s=0.5, alpha=1)

                # Axis formatting
                ax_temp.set_box_aspect([xlen_t, ylen_t, zlen_t])
                ax_temp.set_title(f"{SPACECRAFT} - Temporal ATHARV plot - {EVENT}")
                ax_temp.set_xlabel('$-R$', labelpad=100)
                ax_temp.set_ylabel('$T$')
                ax_temp.set_zlabel('$N$')
                ax_temp.set_yticklabels([])
                ax_temp.set_zticklabels([])
                ax_temp.set_xlim(xlim_t[0], xlim_t[1])
                ax_temp.set_ylim(ylim_t[0], ylim_t[1])
                ax_temp.set_zlim(zlim_t[0], zlim_t[1])

                locator = mdates.AutoDateLocator(maxticks=10)
                formatter = mdates.DateFormatter('%Y-%m-%d\n%H:%M')
                ax_temp.xaxis.set_major_locator(locator)
                ax_temp.xaxis.set_major_formatter(formatter)
                fig_temp.autofmt_xdate()

                for label in ax_temp.get_xticklabels():
                    label.set_fontsize(8)

                # Colorbars
                sm_mag_temp = ScalarMappable(cmap=cmap_temp, norm=norm_mag_temp)
                sm_mag_temp.set_array([])
                cbar_mag_ax_temp = fig_temp.add_axes([0.85, 0.4, 0.008, 0.3])
                cbar_mag_temp = fig_temp.colorbar(sm_mag_temp, cax=cbar_mag_ax_temp)
                cbar_mag_temp.set_label('$|B|$ (nT)')

                cbar_br_ax_temp = fig_temp.add_axes([0.15, 0.4, 0.008, 0.3])
                cbar_br_temp = fig_temp.colorbar(sc_temp, cax=cbar_br_ax_temp)
                cbar_br_temp.ax.yaxis.set_ticks_position('left')
                cbar_br_temp.ax.yaxis.set_label_position('left')
                cbar_br_temp.set_label(r'$B_{R}$ (nT)')

                if analysis_mode.value == 'CME Event':
                    # Time planes
                    solgaleo.draw_time_plane(ax_temp, X_temp[idx_cme_start], color='blue', alpha=0.2)
                    solgaleo.draw_time_plane(ax_temp, X_temp[idx_mo_start], color='blueviolet', alpha=0.2)
                    solgaleo.draw_time_plane(ax_temp, X_temp[idx_cme_end], color='red', alpha=0.2)

                    for idx, c in [(idx_cme_start, 'k'), (idx_mo_start, 'k'), (idx_cme_end, 'k')]:
                        ax_temp.plot([X_temp[idx], X_temp[idx]], [Y_temp[idx], Y_temp[idx]],
                                    [ax_temp.get_zlim()[0], Z_temp[idx]],
                                    color=c, linestyle='--', linewidth=1, alpha=1)

                    ax_temp.scatter([], [], [], color='blue', label='CME Start', marker='s', alpha=0.2)
                    ax_temp.scatter([], [], [], color='blueviolet', label='MO Start', marker='s', alpha=0.2)
                    ax_temp.scatter([], [], [], color='red', label='CME End', marker='s', alpha=0.2)
                    ax_temp.legend(loc='upper left')

                fig_temp.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.0)

                plt.show()
                
                #print("  ✅ Temporal plot displayed (you can interact with it above)")

                if SAVE_MP4:
                    print("  💾 Saving MP4...")
                    def update_temporal(angle):
                        ax_temp.view_init(elev=20, azim=-60 + angle)
                        return fig_temp,

                    ani_temporal = FuncAnimation(fig_temp, update_temporal, frames=range(0, 360, 2),
                                                interval=50, blit=True)
                    temp_file_temporal = f"{EVENT}_{SPACECRAFT}-temporal_ATHARV_temp.mp4"
                    final_file_temporal = f"{EVENT}_{SPACECRAFT}-temporal_ATHARV.mp4"
                    import matplotlib as mpl
                    mpl.rcParams['animation.ffmpeg_args'] = ['-loglevel', 'error']

                    ani_temporal.save(
                        temp_file_temporal,
                        writer="ffmpeg"
                    )

                    subprocess.run(
                        [
                            'ffmpeg', '-loglevel', 'error',
                            '-i', temp_file_temporal,
                            '-vf', 'crop=in_w*0.85:in_h*0.9:in_w*0.075:in_h*0.0',
                            '-c:a', 'copy', final_file_temporal
                        ],
                        stdout=subprocess.DEVNULL,
                        stderr=subprocess.DEVNULL,
                        check=True
                    )

                    os.remove(temp_file_temporal)
                    print(f"  ✅ Saved: {final_file_temporal}")

                    # --- Binder/Voila-safe autoplay + loop embed ---
                    from IPython.display import HTML, display
                    import base64

                    with open(final_file_temporal, "rb") as f:
                        video_b64 = base64.b64encode(f.read()).decode("utf-8")

                    display(HTML(f"""
                    <video width="800"
                        autoplay
                        loop
                        muted
                        playsinline
                        controls>
                    <source src="data:video/mp4;base64,{video_b64}" type="video/mp4">
                    </video>

                    <p>
                    <a download="{final_file_temporal}"
                        href="data:video/mp4;base64,{video_b64}">
                        ⬇️ Download MP4
                    </a>
                    </p>
                    """))

                    #plt.close(fig)


                else:
                    pass  # Keep figure open for interaction

            # ============================================================================
            # HODOGRAM (CHINESE FAN)
            # ============================================================================
            if plot_hodogram.value:
                print("\n🎨 Creating hodogram...")

                B_R = B_rtn[:, 0]
                B_T = B_rtn[:, 1]
                B_N = B_rtn[:, 2]

                if t_B and t_B[0].tzinfo is None:
                    t_B = [t.replace(tzinfo=T_CME_START.tzinfo) for t in t_B]

                mask_cme = np.array([(T_CME_START <= t <= T_CME_END) for t in t_B], dtype=bool)
                t_num = mdates.date2num(t_B)

                idx_cme_start_hodo = np.argmin([abs((t - T_CME_START).total_seconds()) for t in t_B])
                idx_cme_end_hodo = np.argmin([abs((t - T_CME_END).total_seconds()) for t in t_B])
                if analysis_mode.value == 'CME Event':
                    idx_mo_start_hodo = np.argmin([abs((t - T_MO_START).total_seconds()) for t in t_B])

                norm_hodo = plt.Normalize(vmin=t_num[idx_cme_start_hodo], vmax=t_num[idx_cme_end_hodo])
                cmap_hodo = plt.cm.plasma_r

                fig_hodo, ax_hodo = plt.subplots(figsize=(12, 8))
                divider = make_axes_locatable(ax_hodo)
                cax_hodo = divider.append_axes("right", size="5%", pad=0.1)

                # Polar grid
                n_theta = 12
                thetas = np.linspace(0, 2*np.pi, n_theta, endpoint=False)
                lim = max(abs(B_T).max(), abs(B_N).max()) * 1.1
                r_max = lim * 2
                r_circles = np.linspace(r_max/4, r_max, 6)

                for theta in thetas:
                    ax_hodo.plot([0, r_max*np.cos(theta)], [0, r_max*np.sin(theta)],
                                color="lightgrey", linestyle="-", linewidth=0.5, zorder=0)

                circle = np.linspace(0, 2*np.pi, 300)
                for r in r_circles:
                    ax_hodo.plot(r*np.cos(circle), r*np.sin(circle),
                                color="lightgrey", linestyle="-", linewidth=0.5, zorder=0)

                # Grey trajectory outside CME
                ax_hodo.plot(np.array(B_T)[~mask_cme], np.array(B_N)[~mask_cme],
                            color="lightgrey", alpha=0.7, zorder=1, linewidth=0.8)

                # Colored line segments for CME interval
                points = np.array([B_T, B_N]).T.reshape(-1, 1, 2)
                segments = np.concatenate([points[:-1], points[1:]], axis=1)

                mask_segments = mask_cme[:-1] & mask_cme[1:]
                segments_cme = segments[mask_segments]
                colors_cme = t_num[:-1][mask_segments]

                lc = LineCollection(segments_cme, cmap=cmap_hodo, norm=norm_hodo)
                lc.set_array(colors_cme)
                lc.set_linewidth(0.8)
                ax_hodo.add_collection(lc)

                sc_hodo = ax_hodo.scatter(np.array(B_T)[mask_cme], np.array(B_N)[mask_cme],
                                        c=t_num[mask_cme], cmap=cmap_hodo, norm=norm_hodo, 
                                        s=3, zorder=3)

                if analysis_mode.value == 'CME Event':
                    ax_hodo.scatter(B_T[idx_mo_start_hodo], B_N[idx_mo_start_hodo],
                                s=80, marker="*", color="gold", edgecolor="black", 
                                zorder=4, label="MO Start")

                # Colorbar
                cbar_hodo = fig_hodo.colorbar(lc, cax=cax_hodo)
                cbar_hodo.set_label("Time (UTC)", labelpad=15)
                cbar_hodo.ax.yaxis.set_major_formatter(mdates.DateFormatter("%m-%d %H:%M"))

                ax_hodo.set_xlabel(r"$B_T$ (nT)")
                ax_hodo.set_ylabel(r"$B_N$ (nT)")
                ax_hodo.set_title(f"{SPACECRAFT} - $B_N$ vs $B_T$ - {EVENT}")
                ax_hodo.grid(False)
                ax_hodo.set_xlim(-lim, lim)
                ax_hodo.set_ylim(-lim, lim)
                ax_hodo.set_aspect("equal", adjustable="box")

                if SAVE_MP4:
                    fig_hodo.savefig(f"{EVENT}_{SPACECRAFT}-sensu.png", dpi=300, bbox_inches="tight")

                plt.show()  # Display the plot
                
                #print("  ✅ Hodogram displayed")

            # ============================================================================
            # ANGLE vs TIME PLOT
            # ============================================================================
            if plot_angle.value:
                print("\n🎨 Creating angle vs time plot...")

                angles = np.unwrap(np.arctan2(B_N, B_T))
                angles = np.degrees(angles)
                magnitudes = np.sqrt(B_R**2 + B_T**2 + B_N**2)

                fig_angle, ax_angle = plt.subplots(figsize=(12, 8))

                sc_angle = ax_angle.scatter(t_B, angles, c=magnitudes, cmap="viridis", s=5)

                if analysis_mode.value == 'CME Event':
                    ax_angle.axvline(T_CME_START, color="k", linestyle="-", linewidth=0.5, label="CME Start")
                    ax_angle.axvline(T_MO_START, color="k", linestyle="-", linewidth=0.5, label="MO Start")
                    ax_angle.axvline(T_CME_END, color="k", linestyle="-", linewidth=0.5, label="CME End")

                cbar_angle = fig_angle.colorbar(sc_angle, ax=ax_angle, label="|B| (nT)", pad=0.02)

                ax_angle.xaxis.set_major_formatter(mdates.DateFormatter("%m-%d %H:%M"))
                fig_angle.autofmt_xdate()

                ax_angle.set_ylabel("Angle with $T$ axis (degrees, unwrapped)")
                ax_angle.set_xlabel("Time (UTC)")
                ax_angle.set_title(f"{SPACECRAFT} - {EVENT}")

                ymin, ymax = ax_angle.get_ylim()
                yticks = np.arange(np.floor(ymin/30)*30, np.ceil(ymax/30)*30 + 1, 30)
                ax_angle.set_yticks(yticks)
                ax_angle.grid(True, which="both", linestyle="-", linewidth=0.5, alpha=0.7)

                if SAVE_MP4:
                    fig_angle.savefig(f"{EVENT}_{SPACECRAFT}-angle_vs_time.png", dpi=300, bbox_inches="tight")
                
                plt.show()  # Display the plot
                
                #print("  ✅ Angle vs time plot displayed")
            
            print("\n" + "="*70)
            print("✨ ALL VISUALIZATIONS COMPLETE!")
            print("="*70)
            
        except FileNotFoundError as e:
            print(f"❌ Error: Could not find file - {e}")
            print("Make sure all data files are in the working directory!")
        except Exception as e:
            print(f"❌ Error: {e}")
            import traceback
            traceback.print_exc()

# Connect button to function
run_button.on_click(run_visualization)

# Info Box HTML
info_html = """
<div style="
    background-color: #ffffff;
    color: #0d2b45;
    padding: 20px;
    border-radius: 10px;
    margin: 15px 0;
    border-left: 6px solid #1e88e5;
    box-shadow: 0 2px 6px rgba(0,0,0,0.08);
">
    <h2 style="margin-top: 0; color: #0b3c6f;">
        📊 ATHARV - CME Visualization Tool
    </h2>

    <p><strong>Instructions:</strong></p>
    <ol>
        <li>Configure parameters below</li>
        <li>Click <strong>Generate Visualizations</strong></li>
    </ol>

    <p>
        <strong>Outputs:</strong>
        Spatial ATHARV, Temporal ATHARV, Hodogram, Angle vs Time
    </p>
</div>
"""

# Layout
config_box = widgets.VBox([
    widgets.HTML("<h2>⚙️ Configuration</h2>"),
    event_name, 
    spacecraft,
    analysis_mode,
    time_widgets_container,
    validation_output,
    widgets.HTML("<br><h3>📊 Plot Selection</h3>"),
    plot_spatial, 
    remapping_container,
    plot_temporal, 
    plot_hodogram, 
    plot_angle,
    widgets.HTML("<br><h3>💾 Export</h3>"),
    save_video,
    widgets.HTML("<br>"),
    run_button,
    output
], layout=widgets.Layout(padding='20px'))

# Display everything
display(HTML(info_html))
config_box  # This will be the cell output
